In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder

Carregar o DataFrame Original e Realizar a Limpeza Inicial

In [2]:
df_modelo = pd.read_csv('/mnt/c/Users/AcerGamer/Downloads/Tuberculose/Projeto-Tuberculose/dados/arquivos_csv/dados_tuberculose_transformados.csv', low_memory=False)

In [3]:
print(f"Shape original: {df_modelo.shape}")

Shape original: (502271, 83)


In [4]:
essential_columns_list = [
    'FORMA',            # Variável alvo
    'CS_SEXO',
    'CS_RACA',
    'NU_IDADE_N',       # Coluna de idade
    'HIV',
    'AGRAVAIDS',
    'AGRAVALCOO',
    'RAIOX_TORA',
    'BACILOSC_E',
    'CULTURA_ES',
    'NU_ANO'
]

In [5]:
# Verificar quais colunas essenciais NÃO estão no DataFrame original
missing_cols_in_original = [col for col in essential_columns_list if col not in df_modelo.columns]

# Selecionar APENAS as colunas essenciais do DataFrame original
df_modelo_cleaned = df_modelo[essential_columns_list].copy()


In [6]:
print(f"\nShape do DataFrame original após seleção das colunas essenciais: {df_modelo_cleaned.shape}")
print(f"Colunas no df_modelo_cleaned: {df_modelo_cleaned.columns.tolist()}")



Shape do DataFrame original após seleção das colunas essenciais: (502271, 11)
Colunas no df_modelo_cleaned: ['FORMA', 'CS_SEXO', 'CS_RACA', 'NU_IDADE_N', 'HIV', 'AGRAVAIDS', 'AGRAVALCOO', 'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES', 'NU_ANO']


In [7]:
# Exibir os tipos de dados e as primeiras linhas do DataFrame limpo
print("\nPrimeiras linhas do df_modelo_cleaned:")
print(df_modelo_cleaned.head())
print("\nTipos de dados no df_modelo_cleaned:")
print(df_modelo_cleaned.dtypes)


Primeiras linhas do df_modelo_cleaned:
   FORMA CS_SEXO   CS_RACA  NU_IDADE_N            HIV  AGRAVAIDS  AGRAVALCOO  \
0    1.0       F     Parda      4011.0  Não realizado        2.0         2.0   
1    1.0       M  Indígena      4046.0   Não Reagente        2.0         2.0   
2    1.0       F     Parda      4065.0  Não realizado        2.0         1.0   
3    1.0       M    Branca      4059.0   Não Reagente        2.0         1.0   
4    1.0       M     Parda      4035.0   Não Reagente        2.0         2.0   

   RAIOX_TORA  BACILOSC_E  CULTURA_ES  NU_ANO  
0         1.0         1.0         4.0    2020  
1         4.0         1.0         4.0    2020  
2         1.0         1.0         4.0    2020  
3         1.0         3.0         2.0    2020  
4         1.0         1.0         4.0    2021  

Tipos de dados no df_modelo_cleaned:
FORMA         float64
CS_SEXO        object
CS_RACA        object
NU_IDADE_N    float64
HIV            object
AGRAVAIDS     float64
AGRAVALCOO    float64

In [8]:
# Verificar NaNs nas colunas essenciais no df_modelo_cleaned (elas podem ter NaNs, mas não serão dropadas aqui)
print("\nContagem de NaNs nas colunas essenciais do df_modelo_cleaned:")
print(df_modelo_cleaned.isnull().sum())


Contagem de NaNs nas colunas essenciais do df_modelo_cleaned:
FORMA          188
CS_SEXO          0
CS_RACA       9585
NU_IDADE_N     203
HIV           1753
AGRAVAIDS     9088
AGRAVALCOO    8612
RAIOX_TORA    9827
BACILOSC_E     166
CULTURA_ES     166
NU_ANO           0
dtype: int64


In [9]:
# Determinar o número de registros saudáveis a serem gerados
df_modelo_cleaned['FORMA'] = df_modelo_cleaned['FORMA'].astype(str) # Garantir tipo string
num_saudaveis = df_modelo_cleaned['FORMA'].value_counts().max()
print(f"\nVolume de dados de pessoas saudáveis a ser gerado: {num_saudaveis} registros.")

# Definir as colunas para make_classification
# Serão TODAS as colunas essenciais, EXCETO a coluna 'FORMA' (que é o target)
cols_to_generate_synth = [col for col in essential_columns_list if col != 'FORMA']

n_features = len(cols_to_generate_synth)
print(f"\nNúmero de features para make_classification: {n_features}")
print(f"Features a serem geradas ou preenchidas pelo make_classification/lógica: {cols_to_generate_synth}")


Volume de dados de pessoas saudáveis a ser gerado: 429028 registros.

Número de features para make_classification: 10
Features a serem geradas ou preenchidas pelo make_classification/lógica: ['CS_SEXO', 'CS_RACA', 'NU_IDADE_N', 'HIV', 'AGRAVAIDS', 'AGRAVALCOO', 'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES', 'NU_ANO']


In [10]:
# Gerar dados sintéticos brutos
X_synthetic, y_synthetic = make_classification(
    n_samples=num_saudaveis,
    n_features=n_features,
    n_informative=min(n_features, 5), # Ajuste conforme a complexidade desejada
    n_redundant=0,
    n_repeated=0,
    n_classes=2,
    random_state=42
)
# Criar um DataFrame a partir dos dados gerados, usando os nomes de colunas definidos
df_saudaveis_raw = pd.DataFrame(X_synthetic, columns=cols_to_generate_synth)

print("\nDataFrame Sintético Bruto (make_classification) - Primeiras linhas:")
print(df_saudaveis_raw.head())
print("\nTipos de dados no df_saudaveis_raw:")
print(df_saudaveis_raw.dtypes)


DataFrame Sintético Bruto (make_classification) - Primeiras linhas:
    CS_SEXO   CS_RACA  NU_IDADE_N       HIV  AGRAVAIDS  AGRAVALCOO  \
0  0.286060  0.265546    1.525834  0.402937   0.815303    0.180180   
1  1.719133 -0.520265   -2.581182  1.073944  -1.308615    0.241425   
2 -0.921168  0.990471   -1.203175  0.291239  -1.135232   -1.085481   
3  0.174429  0.351887   -0.784545 -1.167986   0.859734   -0.288086   
4 -0.423427 -1.160867    0.689477  1.697849  -0.195856   -0.178914   

   RAIOX_TORA  BACILOSC_E  CULTURA_ES    NU_ANO  
0    0.839867    0.564919    0.508709  1.149244  
1    2.123474   -0.665958   -0.122114 -0.649517  
2    1.478781    0.441345    2.667672  1.182332  
3   -1.285834    1.531834    0.673118  0.737098  
4    0.605965   -0.053301    0.475127  1.103795  

Tipos de dados no df_saudaveis_raw:
CS_SEXO       float64
CS_RACA       float64
NU_IDADE_N    float64
HIV           float64
AGRAVAIDS     float64
AGRAVALCOO    float64
RAIOX_TORA    float64
BACILOSC_E    float

In [11]:
# Crie uma cópia para trabalhar e evitar SettingWithCopyWarning
df_saudaveis_mc = df_saudaveis_raw.copy()

# --- Preencher/Gerar as colunas essenciais com a lógica de "Saudável" ---
# Itere sobre todas as colunas que deveriam estar no df_saudaveis_mc (exceto 'FORMA')
for col in cols_to_generate_synth:
    # Se a coluna não foi gerada pelo make_classification (porque n_informative era baixo, etc.)
    # adicione-a ao df_saudaveis_mc antes de tentar preencher.
    if col not in df_saudaveis_mc.columns:
        df_saudaveis_mc[col] = np.nan # Temporariamente NaN, será preenchida abaixo

    if col == 'BACILOSC_E':
        df_saudaveis_mc[col] = 'Negativo'
    elif col == 'CULTURA_ES':
        df_saudaveis_mc[col] = 'Negativo'
    elif col == 'AGRAVAIDS':
        df_saudaveis_mc[col] = '2' # Verifique o valor para "não agravado" no seu dataset
    elif col == 'AGRAVALCOO':
        df_saudaveis_mc[col] = '2' # Verifique o valor para "não agravado" no seu dataset
    elif col == 'CS_SEXO':
        df_saudaveis_mc[col] = np.random.choice(['M', 'F'], size=num_saudaveis, p=[0.5, 0.5]).astype(str)
    elif col == 'HIV':
        df_saudaveis_mc[col] = np.random.choice(['Negativo', 'Não Informado'], size=num_saudaveis, p=[0.9, 0.1]).astype(str)
    elif col == 'RAIOX_TORA':
        df_saudaveis_mc[col] = np.random.choice(['Normal', 'Não Realizado'], size=num_saudaveis, p=[0.8, 0.2]).astype(str)
    elif col == 'CS_RACA':
        df_saudaveis_mc[col] = np.random.choice(['1.0', '2.0', '3.0', '4.0', '5.0'],
                                                size=num_saudaveis,
                                                p=[0.4, 0.3, 0.15, 0.1, 0.05]).astype(str)
    elif col == 'NU_IDADE_N': # Lógica para idade
        min_age, max_age = 0, 100
        # Normalizar os valores gerados para cair no range desejado
        # Se make_classification gerou essa coluna, use os valores gerados e mapeie.
        # Se não gerou (porque não estava em n_informative), apenas random_randint
        if col in df_saudaveis_raw.columns and \
           (df_saudaveis_raw[col].max() - df_saudaveis_raw[col].min()) > 0:
            df_saudaveis_mc[col] = ((df_saudaveis_raw[col] - df_saudaveis_raw[col].min()) /
                                    (df_saudaveis_raw[col].max() - df_saudaveis_raw[col].min())) * (max_age - min_age) + min_age
        else: # Se a coluna não foi gerada ou todos os valores são iguais, use random int
            df_saudaveis_mc[col] = np.random.randint(min_age, max_age + 1, size=num_saudaveis)
        df_saudaveis_mc[col] = df_saudaveis_mc[col].astype(int)

    elif col == 'NU_ANO': # Lógica para ano
        min_year, max_year = 2020, 2024
        if col in df_saudaveis_raw.columns and \
           (df_saudaveis_raw[col].max() - df_saudaveis_raw[col].min()) > 0:
            df_saudaveis_mc[col] = ((df_saudaveis_raw[col] - df_saudaveis_raw[col].min()) /
                                    (df_saudaveis_raw[col].max() - df_saudaveis_raw[col].min())) * (max_year - min_year) + min_year
        else: # Se a coluna não foi gerada ou todos os valores são iguais, use random int
            df_saudaveis_mc[col] = np.random.randint(min_year, max_year + 1, size=num_saudaveis)
        df_saudaveis_mc[col] = df_saudaveis_mc[col].astype(int)

In [12]:
# Adicionar a coluna 'FORMA' com o label para "Saudável"
df_saudaveis_mc['FORMA'] = '4.0' # Use o mesmo tipo (string) que você usou para a coluna 'FORMA' no df_modelo_cleaned

print("\nDataFrame de Pessoas Saudáveis (make_classification + ajustes) - Primeiras linhas:")
print(df_saudaveis_mc.head())
print("\nTipos de dados no df_saudaveis_mc:")
print(df_saudaveis_mc.dtypes)
print(f"\nVolume final de dados de pessoas saudáveis: {len(df_saudaveis_mc)} registros.")



DataFrame de Pessoas Saudáveis (make_classification + ajustes) - Primeiras linhas:
  CS_SEXO CS_RACA  NU_IDADE_N       HIV AGRAVAIDS AGRAVALCOO RAIOX_TORA  \
0       M     1.0          66  Negativo         2          2     Normal   
1       F     1.0          36  Negativo         2          2     Normal   
2       M     1.0          46  Negativo         2          2     Normal   
3       F     1.0          49  Negativo         2          2     Normal   
4       M     5.0          60  Negativo         2          2     Normal   

  BACILOSC_E CULTURA_ES  NU_ANO FORMA  
0   Negativo   Negativo    2022   4.0  
1   Negativo   Negativo    2021   4.0  
2   Negativo   Negativo    2022   4.0  
3   Negativo   Negativo    2022   4.0  
4   Negativo   Negativo    2022   4.0  

Tipos de dados no df_saudaveis_mc:
CS_SEXO       object
CS_RACA       object
NU_IDADE_N     int64
HIV           object
AGRAVAIDS     object
AGRAVALCOO    object
RAIOX_TORA    object
BACILOSC_E    object
CULTURA_ES    object


In [13]:
# Verificar se todas as colunas essenciais estão no df_saudaveis_mc
missing_in_synth = [col for col in essential_columns_list if col not in df_saudaveis_mc.columns]
if missing_in_synth:
    print(f"\nATENÇÃO: As seguintes colunas essenciais ainda estão ausentes no df_saudaveis_mc: {missing_in_synth}")
else:
    print("\nTodas as colunas essenciais estão presentes no df_saudaveis_mc.")



Todas as colunas essenciais estão presentes no df_saudaveis_mc.


In [14]:
# --- Alinhar as colunas antes da concatenação ---
cols_for_final_df = essential_columns_list
df_saudaveis_mc_adjusted = df_saudaveis_mc.reindex(columns=cols_for_final_df)

In [15]:
df_treino_final = pd.concat([df_modelo_cleaned, df_saudaveis_mc_adjusted], ignore_index=True)

print("\n--- DataFrame Final para Treinamento (df_treino_final) info ---")
print(df_treino_final.info())
print("\nContagem de 'FORMA' no DataFrame final:")
print(df_treino_final['FORMA'].value_counts())



--- DataFrame Final para Treinamento (df_treino_final) info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931299 entries, 0 to 931298
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FORMA       931299 non-null  object 
 1   CS_SEXO     931299 non-null  object 
 2   CS_RACA     921714 non-null  object 
 3   NU_IDADE_N  931096 non-null  float64
 4   HIV         929546 non-null  object 
 5   AGRAVAIDS   922211 non-null  object 
 6   AGRAVALCOO  922687 non-null  object 
 7   RAIOX_TORA  921472 non-null  object 
 8   BACILOSC_E  931133 non-null  object 
 9   CULTURA_ES  931133 non-null  object 
 10  NU_ANO      931299 non-null  int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 78.2+ MB
None

Contagem de 'FORMA' no DataFrame final:
FORMA
1.0    429028
4.0    429028
2.0     56767
3.0     16288
nan       188
Name: count, dtype: int64


In [16]:
# Verificar nulos novamente, nas colunas presentes no DataFrame final.
print("\nVerificação de nulos nas colunas presentes no DataFrame final:")
nans_per_column = df_treino_final.isnull().sum()
nans_per_column = nans_per_column[nans_per_column > 0] # Mostra apenas colunas com NaNs

if not nans_per_column.empty:
    print(nans_per_column)
else:
    print("Nenhuma coluna presente no DataFrame final possui valores nulos.")



Verificação de nulos nas colunas presentes no DataFrame final:
CS_RACA       9585
NU_IDADE_N     203
HIV           1753
AGRAVAIDS     9088
AGRAVALCOO    8612
RAIOX_TORA    9827
BACILOSC_E     166
CULTURA_ES     166
dtype: int64


In [17]:
df_treino_final.to_csv('df_treino_final_para_modelo.csv', index=False)
print("\nDataFrame final com pessoas saudáveis salvo em 'df_treino_final_para_modelo.csv'")


DataFrame final com pessoas saudáveis salvo em 'df_treino_final_para_modelo.csv'


# ignora

In [4]:
# Definir o threshold para remoção de colunas com muitos valores nulos (30%)
threshold = 0.30 * len(df_modelo)

# Aplicar a limpeza de colunas com mais de 30% de NaN no DataFrame original
df_modelo_cleaned = df_modelo.dropna(axis=1, thresh=threshold)

print(f"\nShape do DataFrame original após remoção de colunas com >30% NaN: {df_modelo_cleaned.shape}")
print(f"Colunas removidas: {list(set(df_modelo.columns) - set(df_modelo_cleaned.columns))}")

# Exibir os tipos de dados e as primeiras linhas do DataFrame limpo
print("\nPrimeiras linhas do df_modelo_cleaned:")
print(df_modelo_cleaned.head())
print("\nTipos de dados no df_modelo_cleaned:")
print(df_modelo_cleaned.dtypes)


Shape do DataFrame original após remoção de colunas com >30% NaN: (502271, 60)
Colunas removidas: ['DT_TRANSUS', 'DOENCA_TRA', 'BACILOSC_O', 'EXTRAPU1_N', 'CULTURA_OU', 'EXTRAPU2_N', 'DT_TRANSRS', 'EXTRAPUL_O', 'ANT_RETRO', 'AGRAVOUTDE', 'NDUPLIC_N', 'INSTITUCIO', 'BAC_APOS_6', 'TESTE_TUBE', 'TRAT_SUPER', 'IN_VINCULA', 'BACILOS_E2', 'MUN_TRANSF', 'TRANSF', 'NU_IDADE_ANOS', 'SITUA_9_M', 'DT_TRANSDM', 'UF_TRANSF']

Primeiras linhas do df_modelo_cleaned:
   TP_NOT ID_AGRAVO                     DT_NOTIFIC  NU_ANO  SG_UF_NOT  \
0       2      A169  1970-01-01 00:00:00.020200930    2020         15   
1       2      A169  1970-01-01 00:00:00.020200710    2020         15   
2       2      A169  1970-01-01 00:00:00.020201203    2020         22   
3       2      A169  1970-01-01 00:00:00.020200210    2020         24   
4       2      A169  1970-01-01 00:00:00.020210114    2021         43   

   ID_MUNICIP  ID_REGIONA                        DT_DIAG  ANO_NASC  \
0      150650       148.0  1970-01

Determinar o Volume de Dados "Saudáveis" e Preparar Listas de Colunas

In [6]:
# Determinar o número de registros saudáveis a serem gerados
# Usando a contagem da maior classe existente no df_modelo_cleaned (assumindo 'FORMA' é a coluna)
# OBS: Garanta que 'FORMA' seja tratada como string para a contagem
df_modelo_cleaned['FORMA'] = df_modelo_cleaned['FORMA'].astype(str)
num_saudaveis = df_modelo_cleaned['FORMA'].value_counts().max()
print(f"\nVolume de dados de pessoas saudáveis a ser gerado: {num_saudaveis} registros.")


Volume de dados de pessoas saudáveis a ser gerado: 429028 registros.


/tmp/ipykernel_635/2847037123.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modelo_cleaned['FORMA'] = df_modelo_cleaned['FORMA'].astype(str)


In [7]:

# Definir as colunas que make_classification irá gerar.
# Estas devem ser as colunas que você quer que tenham dados sintéticos, e que restaram no df_modelo_cleaned.
# Remova 'FORMA' da lista, pois ela será nossa coluna target (label '4.0').
# Remova 'total_comorbidades' e 'hiv_positivo_com_aids' se você vai defini-las com valores fixos.
cols_to_generate_synth = [
    'CS_SEXO', 'CS_RACA', 'HIV', 'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES',
    'NU_ANO', 'NU_IDADE_ANOS_CORRETA', 'faixa_etaria'
]

# Apenas para garantir que estamos usando apenas colunas que sobreviveram à limpeza
cols_to_generate_synth = [col for col in cols_to_generate_synth if col in df_modelo_cleaned.columns]

n_features = len(cols_to_generate_synth)
print(f"\nNúmero de features para make_classification: {n_features}")
print(f"Features a serem geradas pelo make_classification: {cols_to_generate_synth}")


Número de features para make_classification: 7
Features a serem geradas pelo make_classification: ['CS_SEXO', 'CS_RACA', 'HIV', 'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES', 'NU_ANO']


Gerar Dados Sintéticos Brutos

In [8]:
# Gerar dados sintéticos brutos
X_synthetic, y_synthetic = make_classification(
    n_samples=num_saudaveis,
    n_features=n_features,
    n_informative=min(n_features, 5), # Ajuste conforme a complexidade desejada
    n_redundant=0,
    n_repeated=0,
    n_classes=2, # make_classification gera 2 classes, usaremos apenas os dados gerados
    random_state=42
)

# Criar um DataFrame a partir dos dados gerados, usando os nomes de colunas definidos
df_saudaveis_raw = pd.DataFrame(X_synthetic, columns=cols_to_generate_synth)

print("\nDataFrame Sintético Bruto (make_classification) - Primeiras linhas:")
print(df_saudaveis_raw.head())
print("\nTipos de dados no df_saudaveis_raw:")
print(df_saudaveis_raw.dtypes)


DataFrame Sintético Bruto (make_classification) - Primeiras linhas:
    CS_SEXO   CS_RACA       HIV  RAIOX_TORA  BACILOSC_E  CULTURA_ES    NU_ANO
0  1.425149 -0.760239  2.285482   -0.779623   -0.342318   -0.340903  1.939994
1 -1.539600 -0.851875 -2.221309    1.341345   -1.302185    0.685744  3.457359
2  0.163617 -0.699274 -4.668132    1.837927   -0.038641    0.713883  1.657483
3  0.683234  0.138720  0.993088    0.081546   -1.068113   -0.435609  2.146210
4 -1.019916  0.519429  1.630425    0.009991   -0.384413    0.276052  3.411711

Tipos de dados no df_saudaveis_raw:
CS_SEXO       float64
CS_RACA       float64
HIV           float64
RAIOX_TORA    float64
BACILOSC_E    float64
CULTURA_ES    float64
NU_ANO        float64
dtype: object


## Mapear e Aplicar a Lógica de "Saudável" nos Dados Sintéticos

In [9]:
# Crie uma cópia para trabalhar e evitar SettingWithCopyWarning
df_saudaveis_mc = df_saudaveis_raw.copy()

# --- Aplicar a lógica de "inverter padrões" e mapeamento para valores reais ---

# 1. Colunas com valores fixos para "Saudável"
# ATENÇÃO: Verifique se estas colunas EXISTEM no df_modelo_cleaned antes de aplicar!
if 'BACILOSC_E' in df_saudaveis_mc.columns:
    df_saudaveis_mc['BACILOSC_E'] = 'Negativo'
if 'CULTURA_ES' in df_saudaveis_mc.columns:
    df_saudaveis_mc['CULTURA_ES'] = 'Negativo'

# 2. Colunas com valores de escolha (categóricas com poucas opções)
# Use np.random.choice com probabilidades para as categorias
if 'CS_SEXO' in df_saudaveis_mc.columns:
    df_saudaveis_mc['CS_SEXO'] = np.random.choice(['M', 'F'], size=num_saudaveis, p=[0.5, 0.5]).astype(str)
if 'HIV' in df_saudaveis_mc.columns:
    df_saudaveis_mc['HIV'] = np.random.choice(['Negativo', 'Não Informado'], size=num_saudaveis, p=[0.9, 0.1]).astype(str)
if 'RAIOX_TORA' in df_saudaveis_mc.columns:
    df_saudaveis_mc['RAIOX_TORA'] = np.random.choice(['Normal', 'Não Realizado'], size=num_saudaveis, p=[0.8, 0.2]).astype(str)

# Para CS_RACA (se existir), que tem 5 valores, use np.random.choice com probabilidades ajustadas
if 'CS_RACA' in df_saudaveis_mc.columns:
    df_saudaveis_mc['CS_RACA'] = np.random.choice(['1.0', '2.0', '3.0', '4.0', '5.0'],
                                                   size=num_saudaveis,
                                                   p=[0.4, 0.3, 0.15, 0.1, 0.05]).astype(str) # Ajuste as probabilidades


In [10]:
# 3. Colunas numéricas contínuas (idade, ano)
# Normalize os valores gerados para cair em um range desejado
if 'NU_IDADE_ANOS_CORRETA' in df_saudaveis_mc.columns:
    min_age, max_age = 0, 100
    # Evitar divisão por zero se todos os valores forem iguais (improvável com make_classification)
    if df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].max() - df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].min() > 0:
        df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'] = ((df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'] - df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].min()) /
                                                (df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].max() - df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].min())) * (max_age - min_age) + min_age
    else: # Se todos os valores forem iguais, atribua um valor médio
        df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'] = (min_age + max_age) / 2
    df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'] = df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].astype(int)

if 'NU_ANO' in df_saudaveis_mc.columns:
    min_year, max_year = 2010, 2024
    if df_saudaveis_mc['NU_ANO'].max() - df_saudaveis_mc['NU_ANO'].min() > 0:
        df_saudaveis_mc['NU_ANO'] = ((df_saudaveis_mc['NU_ANO'] - df_saudaveis_mc['NU_ANO'].min()) /
                                  (df_saudaveis_mc['NU_ANO'].max() - df_saudaveis_mc['NU_ANO'].min())) * (max_year - min_year) + min_year
    else: # Se todos os valores forem iguais, atribua um valor médio
        df_saudaveis_mc['NU_ANO'] = (min_year + max_year) / 2
    df_saudaveis_mc['NU_ANO'] = df_saudaveis_mc['NU_ANO'].astype(int)


In [11]:
# 4. Colunas derivadas ou com valor fixo para "saudável"
# ATENÇÃO: Verifique se estas colunas EXISTEM no df_modelo_cleaned antes de aplicar!
if 'total_comorbidades' in df_modelo_cleaned.columns and 'total_comorbidades' not in df_saudaveis_mc.columns:
    df_saudaveis_mc['total_comorbidades'] = 0
if 'hiv_positivo_com_aids' in df_modelo_cleaned.columns and 'hiv_positivo_com_aids' not in df_saudaveis_mc.columns:
    df_saudaveis_mc['hiv_positivo_com_aids'] = 'False' # Como você converteu para string antes

# 5. Faixa Etária (derivada da idade, como você provavelmente fez no df_modelo)
# ATENÇÃO: Verifique se 'faixa_etaria' existe ou é criada com base na 'NU_IDADE_ANOS_CORRETA'
if 'faixa_etaria' in df_saudaveis_mc.columns:
    def categorize_age(age):
        if age < 18:
            return 'Jovem'
        elif age <= 35:
            return 'Adulto Jovem'
        elif age <= 50:
            return 'Adulto'
        elif age <= 65:
            return 'Idoso'
        else:
            return 'Muito Idoso'
    df_saudaveis_mc['faixa_etaria'] = df_saudaveis_mc['NU_IDADE_ANOS_CORRETA'].apply(categorize_age).astype(str)


In [12]:
# 6. Adicionar a coluna 'FORMA' com o label para "Saudável"
df_saudaveis_mc['FORMA'] = '4.0' # Use o mesmo tipo (string) que você usou para a coluna 'FORMA' no df_modelo_cleaned

print("\nDataFrame de Pessoas Saudáveis (make_classification + ajustes) - Primeiras linhas:")
print(df_saudaveis_mc.head())
print("\nTipos de dados no df_saudaveis_mc:")
print(df_saudaveis_mc.dtypes)
print(f"\nVolume final de dados de pessoas saudáveis: {len(df_saudaveis_mc)} registros.")


DataFrame de Pessoas Saudáveis (make_classification + ajustes) - Primeiras linhas:
  CS_SEXO CS_RACA            HIV     RAIOX_TORA BACILOSC_E CULTURA_ES  NU_ANO  \
0       M     1.0       Negativo         Normal   Negativo   Negativo    2018   
1       M     2.0       Negativo  Não Realizado   Negativo   Negativo    2020   
2       F     1.0  Não Informado         Normal   Negativo   Negativo    2018   
3       F     1.0       Negativo         Normal   Negativo   Negativo    2018   
4       M     2.0       Negativo  Não Realizado   Negativo   Negativo    2019   

  FORMA  
0   4.0  
1   4.0  
2   4.0  
3   4.0  
4   4.0  

Tipos de dados no df_saudaveis_mc:
CS_SEXO       object
CS_RACA       object
HIV           object
RAIOX_TORA    object
BACILOSC_E    object
CULTURA_ES    object
NU_ANO         int64
FORMA         object
dtype: object

Volume final de dados de pessoas saudáveis: 429028 registros.


Alinhar e Concatenar os DataFrames Finalizados

In [16]:
# --- Alinhar as colunas antes da concatenação ---
# O df_modelo_cleaned já está com as colunas que sobreviveram à limpeza.

# Queremos que df_saudaveis_mc_adjusted contenha APENAS as colunas que estão
# no df_modelo_cleaned, além da nossa coluna 'FORMA'.
# Primeiro, vamos garantir que 'FORMA' está no df_saudaveis_mc (já deve estar da Célula 5)
# e que é do tipo string.
df_saudaveis_mc['FORMA'] = df_saudaveis_mc['FORMA'].astype(str)

# Criar a lista de colunas que o DataFrame FINAL deve ter (as do df_modelo_cleaned)
cols_for_final_df = df_modelo_cleaned.columns.tolist()

# Filtrar o df_saudaveis_mc para conter SOMENTE as colunas que também estão em df_modelo_cleaned.
# Isso vai descartar quaisquer colunas geradas no sintético que não existam no original limpo,
# e, mais importante, não irá introduzir novas colunas cheias de NaN.
df_saudaveis_mc_adjusted = df_saudaveis_mc[df_saudaveis_mc.columns.intersection(cols_for_final_df)]

# Em seguida, reordenar as colunas para que fiquem na mesma ordem do df_modelo_cleaned
# Colunas que estão em df_modelo_cleaned mas NÃO foram geradas no df_saudaveis_mc (e, portanto, não estão em df_saudaveis_mc_adjusted)
# serão preenchidas com NaN QUANDO O REINDEX É APLICADO COM UM NOVO CONJUNTO DE COLUNAS.
# No entanto, a quantidade de NaNs para essas colunas será bem menor do que antes,
# pois elas não virão do sintético, e sim do original, onde já tinham um percentual baixo o suficiente para não serem descartadas.
df_saudaveis_mc_adjusted = df_saudaveis_mc_adjusted.reindex(columns=cols_for_final_df)


In [17]:
# --- Concatenação dos DataFrames ---
df_treino_final = pd.concat([df_modelo_cleaned, df_saudaveis_mc_adjusted], ignore_index=True)

print("\n--- DataFrame Final para Treinamento (df_treino_final) info ---")
print(df_treino_final.info())
print("\nContagem de 'FORMA' no DataFrame final:")
print(df_treino_final['FORMA'].value_counts())

# Verificar nulos novamente, com as colunas que realmente estão no DataFrame final
print("\nVerificação de nulos nas colunas presentes no DataFrame final:")
nans_per_column = df_treino_final.isnull().sum()
nans_per_column = nans_per_column[nans_per_column > 0] # Mostra apenas colunas com NaNs

if not nans_per_column.empty:
    print(nans_per_column)
else:
    print("Nenhuma coluna presente no DataFrame final possui valores nulos.")



--- DataFrame Final para Treinamento (df_treino_final) info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931299 entries, 0 to 931298
Data columns (total 60 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   TP_NOT      502271 non-null  float64
 1   ID_AGRAVO   502271 non-null  object 
 2   DT_NOTIFIC  502271 non-null  object 
 3   NU_ANO      931299 non-null  int64  
 4   SG_UF_NOT   502271 non-null  float64
 5   ID_MUNICIP  502271 non-null  float64
 6   ID_REGIONA  297899 non-null  float64
 7   DT_DIAG     502271 non-null  object 
 8   ANO_NASC    500066 non-null  float64
 9   NU_IDADE_N  502068 non-null  float64
 10  CS_SEXO     931299 non-null  object 
 11  CS_GESTANT  502222 non-null  object 
 12  CS_RACA     921714 non-null  object 
 13  CS_ESCOL_N  427367 non-null  object 
 14  SG_UF       502271 non-null  float64
 15  ID_MN_RESI  502271 non-null  float64
 16  ID_RG_RESI  296327 non-null  float64
 17  ID_PAIS     389768 no

In [18]:
# Salvar o DataFrame final para uso no notebook do modelo
df_treino_final.to_csv('df_treino_final_para_modelo.csv', index=False)
print("\nDataFrame final com pessoas saudáveis salvo em 'df_treino_final_para_modelo.csv'")


DataFrame final com pessoas saudáveis salvo em 'df_treino_final_para_modelo.csv'


In [19]:
print(df_treino_final['FORMA'].value_counts())

FORMA
1.0    429028
4.0    429028
2.0     56767
3.0     16288
nan       188
Name: count, dtype: int64


In [3]:
categorical_cols_original = [
    'CS_SEXO', 'CS_RACA', 'HIV', 'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES'
]
for col in categorical_cols_original:
    if col in df_modelo.columns:
        df_modelo[col] = df_modelo[col].astype(str)

if 'hiv_positivo_com_aids' in df_modelo.columns:
    df_modelo['hiv_positivo_com_aids'] = df_modelo['hiv_positivo_com_aids'].astype(str)

In [8]:
if df_modelo['FORMA'].dtype == 'float64':
    # Trate nulos se houver antes de converter para int, se necessário
    df_modelo['FORMA'] = df_modelo['FORMA'].fillna(-1).astype(int) # Exemplo de tratamento de nulos
    df_modelo['FORMA'] = df_modelo['FORMA'].astype(str) # Convertendo para string para consistência com as categóricas

print("DataFrame Original (df_modelo) - Primeiras linhas:")
print(df_modelo.head())
print("\nContagem de valores na coluna 'FORMA' (antes da geração):")
print(df_modelo['FORMA'].value_counts())
print("\nTipos de dados no df_modelo:")
print(df_modelo.dtypes)

DataFrame Original (df_modelo) - Primeiras linhas:
   TP_NOT ID_AGRAVO                     DT_NOTIFIC  NU_ANO  SG_UF_NOT  \
0       2      A169  1970-01-01 00:00:00.020200930    2020         15   
1       2      A169  1970-01-01 00:00:00.020200710    2020         15   
2       2      A169  1970-01-01 00:00:00.020201203    2020         22   
3       2      A169  1970-01-01 00:00:00.020200210    2020         24   
4       2      A169  1970-01-01 00:00:00.020210114    2021         43   

   ID_MUNICIP  ID_REGIONA                        DT_DIAG  ANO_NASC  \
0      150650       148.0  1970-01-01 00:00:00.020200930    2009.0   
1      150530       149.0  1970-01-01 00:00:00.020200707    1973.0   
2      220770       186.0  1970-01-01 00:00:00.020201119    1955.0   
3      240920       141.0  1970-01-01 00:00:00.020200208    1960.0   
4      430210       160.0  1970-01-01 00:00:00.020201226    1984.0   

   NU_IDADE_N  ... TEST_MOLEC TEST_SENSI ANT_RETRO BAC_APOS_6  TRANSF  \
0      4011.0  .

In [9]:
# Pega o tamanho da maior classe de 'FORMA' no df_modelo
num_saudaveis = df_modelo['FORMA'].value_counts().max()
print(f"\nO volume de dados de pessoas saudáveis a ser gerado é: {num_saudaveis} registros.")


O volume de dados de pessoas saudáveis a ser gerado é: 429028 registros.


In [10]:
cols_to_generate = [
    'CS_SEXO', 'CS_RACA', 'HIV', 'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES',
    'NU_ANO', 'NU_IDADE_ANOS_CORRETA', 'faixa_etaria'
]
n_features = len(cols_to_generate)

In [11]:
X_synthetic, y_synthetic = make_classification(
    n_samples=num_saudaveis,
    n_features=n_features,
    n_informative=min(n_features, 5), # Ajuste conforme a complexidade desejada
    n_redundant=0,
    n_repeated=0,
    n_classes=2, # Estamos gerando dados para uma única classe (saudáveis), mas make_classification gera 2.
                 # Vamos usar apenas uma delas ou ignorar o y_synthetic.
    random_state=42
)

In [12]:
df_saudaveis = pd.DataFrame(X_synthetic, columns=[f'feature_{i}' for i in range(n_features)])

In [13]:
print("\nDataFrame Sintético Bruto (make_classification) - Primeiras linhas:")
print(df_saudaveis.head())


DataFrame Sintético Bruto (make_classification) - Primeiras linhas:
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   1.467606   2.324934   2.161475  -3.189477   1.070771  -0.180757   
1   0.771258   1.188837   1.570261  -1.285053   0.999545   0.030869   
2  -0.166884   2.119564   1.313741  -1.056231   1.826371  -1.093788   
3  -1.112893   1.004029   0.294403  -0.129176   0.842375  -0.291284   
4   1.340354   1.963279  -0.639310  -0.919330   1.507229  -1.621297   

   feature_6  feature_7  feature_8  
0  -0.212302   1.119145   0.490310  
1  -1.098832   1.026123   0.002288  
2   0.043393   2.406625   0.718804  
3   0.111744   2.288384   0.621683  
4   1.619151  -0.192253   0.890964  


In [14]:
# Renomeie as colunas para seus nomes significativos
df_saudaveis.columns = cols_to_generate

# --- Aplique a lógica de "inverter padrões" e mapeamento para valores reais ---

# 1. Colunas com valores fixos para "Saudável"
df_saudaveis['BACILOSC_E'] = 'Negativo'
df_saudaveis['CULTURA_ES'] = 'Negativo'

# 2. Colunas com valores de escolha (e.g., binários ou poucos categóricos)
# Use thresholds nos valores numéricos gerados para mapear para suas categorias.
# Exemplo para CS_SEXO (assumindo que feature_0 foi CS_SEXO):
# Use uma distribuição aproximada do que você espera para pessoas saudáveis
df_saudaveis['CS_SEXO'] = np.random.choice(['M', 'F'], size=num_saudaveis, p=[0.5, 0.5]).astype(str) # 50/50
df_saudaveis['HIV'] = np.random.choice(['Negativo', 'Não Informado'], size=num_saudaveis, p=[0.9, 0.1]).astype(str)
df_saudaveis['RAIOX_TORA'] = np.random.choice(['Normal', 'Não Realizado'], size=num_saudaveis, p=[0.8, 0.2]).astype(str)

# Para CS_RACA, que tem 5 valores, você pode usar np.random.choice com probabilidades ajustadas
df_saudaveis['CS_RACA'] = np.random.choice(['1.0', '2.0', '3.0', '4.0', '5.0'],
                                               size=num_saudaveis,
                                               p=[0.4, 0.3, 0.15, 0.1, 0.05]).astype(str) # Ajuste as probabilidades


In [15]:
# 3. Colunas numéricas contínuas (idade, ano)
min_age, max_age = 0, 120
df_saudaveis['NU_IDADE_ANOS_CORRETA'] = ((df_saudaveis['NU_IDADE_ANOS_CORRETA'] - df_saudaveis['NU_IDADE_ANOS_CORRETA'].min()) /
                                            (df_saudaveis['NU_IDADE_ANOS_CORRETA'].max() - df_saudaveis['NU_IDADE_ANOS_CORRETA'].min())) * (max_age - min_age) + min_age
df_saudaveis['NU_IDADE_ANOS_CORRETA'] = df_saudaveis['NU_IDADE_ANOS_CORRETA'].astype(int)

# NU_ANO:
min_year, max_year = 2010, 2024
df_saudaveis['NU_ANO'] = ((df_saudaveis['NU_ANO'] - df_saudaveis['NU_ANO'].min()) /
                              (df_saudaveis['NU_ANO'].max() - df_saudaveis['NU_ANO'].min())) * (max_year - min_year) + min_year
df_saudaveis['NU_ANO'] = df_saudaveis['NU_ANO'].astype(int)


In [16]:
# 4. Colunas derivadas ou com valor fixo para "saudável"
df_saudaveis['total_comorbidades'] = 0
df_saudaveis['hiv_positivo_com_aids'] = 'False' # Certifique-se de que é string


In [17]:
# 5. Faixa Etária (derivada da idade, como você provavelmente fez no df_modelo)
def categorize_age(age):
    if age < 18:
        return 'Jovem' # Ou defina um valor padrão, ou reajuste idades
    elif age <= 35:
        return 'Adulto Jovem'
    elif age <= 50:
        return 'Adulto'
    elif age <= 65:
        return 'Idoso'
    else:
        return 'Muito idoso' # Ou defina um valor padrão, ou reajuste idades
df_saudaveis['faixa_etaria'] = df_saudaveis['NU_IDADE_ANOS_CORRETA'].apply(categorize_age).astype(str)


In [18]:
# 6. Adicionar a coluna 'FORMA' com o label para "Saudável"
df_saudaveis['FORMA'] = '4.0' # Use o mesmo tipo (string) que você usou para a coluna 'FORMA' no df_modelo
# Reordene as colunas para que fiquem iguais ao df_modelo, se for concatenar
# Obtenha as colunas do df_modelo para garantir a ordem
common_cols = [col for col in df_modelo.columns if col in df_saudaveis.columns]
df_saudaveis = df_saudaveis[common_cols]

In [19]:
print("\nDataFrame de Pessoas Saudáveis (make_classification + ajustes) - Primeiras linhas:")
print(df_saudaveis.head())
print("\nTipos de dados no df_saudaveis:")
print(df_saudaveis.dtypes)
print(f"\nVolume final de dados de pessoas saudáveis: {len(df_saudaveis)} registros.")


DataFrame de Pessoas Saudáveis (make_classification + ajustes) - Primeiras linhas:
   NU_ANO CS_SEXO CS_RACA     RAIOX_TORA FORMA BACILOSC_E CULTURA_ES  \
0    2016       M     1.0         Normal   4.0   Negativo   Negativo   
1    2014       M     2.0  Não Realizado   4.0   Negativo   Negativo   
2    2016       F     1.0  Não Realizado   4.0   Negativo   Negativo   
3    2016       M     2.0         Normal   4.0   Negativo   Negativo   
4    2018       F     3.0         Normal   4.0   Negativo   Negativo   

        HIV  NU_IDADE_ANOS_CORRETA faixa_etaria  
0  Negativo                     64        Idoso  
1  Negativo                     63        Idoso  
2  Negativo                     76  Muito idoso  
3  Negativo                     75  Muito idoso  
4  Negativo                     52        Idoso  

Tipos de dados no df_saudaveis:
NU_ANO                    int64
CS_SEXO                  object
CS_RACA                  object
RAIOX_TORA               object
FORMA                 

In [20]:
# Identifique colunas que estão apenas em um dos dataframes
cols_in_model_only = list(set(df_modelo.columns) - set(df_saudaveis.columns))
cols_in_synthetic_only = list(set(df_saudaveis.columns) - set(df_modelo.columns))

In [21]:
# Adicione as colunas que faltam em df_saudaveis_mc
for col in cols_in_model_only:
    df_saudaveis[col] = np.nan # Ou um valor padrão apropriado

# Adicione as colunas que faltam em df_modelo (se houver, o que é menos provável para o original)
for col in cols_in_synthetic_only:
    df_modelo[col] = np.nan # Ou um valor padrão apropriado

# Garanta que a ordem das colunas seja a mesma
df_saudaveis = df_saudaveis[df_modelo.columns]

df_final_treino = pd.concat([df_modelo, df_saudaveis], ignore_index=True)

In [22]:
print("\nDataFrame Final para Treinamento (Original + Saudáveis) - Primeiras linhas:")
print(df_final_treino.head())
print(f"Volume total do DataFrame para treinamento: {len(df_final_treino)} registros.")
print("\nContagem de valores na coluna 'FORMA' no DataFrame final:")
print(df_final_treino['FORMA'].value_counts())


DataFrame Final para Treinamento (Original + Saudáveis) - Primeiras linhas:
   TP_NOT ID_AGRAVO                     DT_NOTIFIC  NU_ANO  SG_UF_NOT  \
0     2.0      A169  1970-01-01 00:00:00.020200930    2020       15.0   
1     2.0      A169  1970-01-01 00:00:00.020200710    2020       15.0   
2     2.0      A169  1970-01-01 00:00:00.020201203    2020       22.0   
3     2.0      A169  1970-01-01 00:00:00.020200210    2020       24.0   
4     2.0      A169  1970-01-01 00:00:00.020210114    2021       43.0   

   ID_MUNICIP  ID_REGIONA                        DT_DIAG  ANO_NASC  \
0    150650.0       148.0  1970-01-01 00:00:00.020200930    2009.0   
1    150530.0       149.0  1970-01-01 00:00:00.020200707    1973.0   
2    220770.0       186.0  1970-01-01 00:00:00.020201119    1955.0   
3    240920.0       141.0  1970-01-01 00:00:00.020200208    1960.0   
4    430210.0       160.0  1970-01-01 00:00:00.020201226    1984.0   

   NU_IDADE_N  ... TEST_MOLEC TEST_SENSI ANT_RETRO BAC_APOS_6  

In [23]:
df_final_treino.to_csv('df_treino_com_saudaveis.csv', index=False)
print("DataFrame final com pessoas saudáveis salvo em 'df_treino_com_saudaveis.csv'")

DataFrame final com pessoas saudáveis salvo em 'df_treino_com_saudaveis.csv'
